# r/CasualConversations Database creation
**Note that this Notebook is not useful, since the method used did not allow for more than about 4,5 thousand submissions to be harvested from Reddit.**

This Jupyter Notebook is meant to create a database from submissions to r/CasualConversations. It first extracts submissions and then saves them into a special database file such that feature can be created from the submissions. This notebook is part of a project that tries to predict the flair of a submission on CasualConversations.

In [1]:
import praw # Source code : https://github.com/praw-dev/praw
            # helpful page: https://praw.readthedocs.io/en/latest/code_overview/praw_models.html

# Initialising a connection to Reddit
reddit = praw.Reddit(user_agent='Doing some stuff', client_id='hir_Veg3Rs-Svw', client_secret="giRkbKknUe4Oy4EcoGYA1N_Y0sA",
                    username='', password='')

In [2]:
# General modules and function(s)
import numpy as np
from datetime import datetime
import re
import os

def utc_time(timestamp):
    '''Changes Unix to UTC time'''
    return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

## Extracting submissions from Reddit
First the submissions have to be extracted from Reddit. However, as Reddit does not want more than 1000 submissions to be extracted from a subreddit, this notebook tries to 'bruteforce' more submissions out of Reddit by using the 'subreddit.random()' method from the 'praw' module.

### Set-up and Functions

In [3]:
if 'all_submission_ids' not in locals():
    all_submissions = []
    all_submission_ids = []
    print('"all_submissions" and "all_submission_ids" have been initialised.')
else:
    print('"all_submissions" and "all_submission_ids" already exist')

"all_submissions" and "all_submission_ids" have been initialised.


In [4]:
def load_submission_ids(load_name='submissions_CasualConversations.txt', mistake_catch=0,
                        submissions_list=all_submissions.copy(), submission_ids_list=all_submission_ids.copy()):
    ''' Function for loading the submission_ids to a text file
    
    =========================== ===============================================
    Attribute                   Description
    =========================== ===============================================
    "load_name"                 Under which name the file should be loaded.
    "mistake_catch"             To prevent exidently saving submissions, the
                                existing text file is checked for the amount of
                                IDs in that file. If this amount is greater 
                                than the amount of IDS in the
                                submission_ids_list, then submission_ids_list
                                won't be loadd. 
                                To overwrite this, mistake_catch has to be
                                equal to the amount of IDs in the text file
                                minus the IDs in submission_ids_list
    "submissions_list"          To which list submissions will be loaded.
    "submission_ids_list"       To which list the IDs will be loaded.
    =========================== ===============================================
    '''    
    if(load_name == '' and mistake_catch == -1):
        all_submissions = []
        submission_ids_list = []
        print('The submissions_list and submission_ids_list have been reset to empty lists.')
        return submissions_list, submission_ids_list 
    
    if(load_name[-4:]!='.txt'):
        load_name += '.txt'
        
    if(os.path.isfile(load_name) == False):
        print('The file to load (' + load_name + ') from does not exist. \n' + 
              'You can try to load from a different file or start with an empty submission_ids_list. \n' +
              'To do the latter, you must set the "load_name" to an empty string and "mistake_catch" to -1'
             )
        return
        
    load_file = open(load_name, 'r')
    
    # mistake prevention
    load_file_list = re.split('\n', load_file.read())
    if(len(submission_ids_list) > len(load_file_list) and mistake_catch != len(submission_ids_list) - len(load_file_list)):
        print('Are you sure that you want to overwrite your submission_ids_list that contians more IDs' +
              ' than the text file that you want to load? \n \n' +
              'If so, then set parameter "mistake_catch" to ' + str(len(submission_ids_list) - len(load_file_list)) +
              ' otherwise your mistake has luckily been prevented \n\n' +
              'IDs in text file : ' + str(len(load_file_list)) + '\n' +
              'IDs in submission_ids_list file : ' + str(len(submission_ids_list))
             )
        load_file.close()
        return
    else:
        # loading the submission_ids_list from the text file and loading submissions from Reddit
        submission_ids_list = [ID for ID in load_file_list]
        submissions_list = []
        for ID in submission_ids_list:
            submissions_list.append(reddit.submission(ID))    
        load_file.close()
        print('Loaded succesfully from', load_name, '!')
        return submissions_list.copy(), submission_ids_list.copy()

In [5]:
def save_submission_ids(save_name='submissions_CasualConversations.txt', 
                        mistake_catch=0, submission_ids_list=all_submission_ids.copy()):
    ''' Function for saving the submission_ids from a text file
    
    =========================== ===============================================
    Attribute                   Description
    =========================== ===============================================
    "save_name"                 Under which name the file should be saved.
    "mistake_catch"             To prevent exidently saving submissions, the
                                existing text file is checked for the amount of
                                IDs in that file. If this amount is greater 
                                than the amount of IDS in the
                                submission_ids_list, then submission_ids_list
                                won't be saved. 
                                To overwrite this, mistake_catch has to be
                                equal to the amount of IDs in the text file
                                minus the IDs in submission_ids_list
    "submission_ids_list"       From which list the IDs will be saved.
    =========================== ===============================================
    '''
    if(save_name[-4:]!='.txt'):
        save_name += '.txt'
        
    save_file = open(save_name, 'w+')
    
    # mistake prevention
    save_file_list = re.split('\n', save_file.read())
    if(len(save_file_list) > len(submission_ids_list) and mistake_catch != len(save_file_list) - len(submission_ids_list)):
        print('Are you sure that you want to overwrite this text file that contians more IDs' +
              ' than the submission_ids_list that you want to save? \n \n' +
              'If so, then set parameter "mistake_catch" to ' + str(len(save_file_list) - len(submission_ids_list)) +
              ' otherwise your mistake has luckily been prevented \n\n' +
              'IDs in text file : ' + str(len(save_file_list)) + '\n' +
              'IDs in submission_ids_list file : ' + str(len(submission_ids_list))
             )
        save_file.close()
        return
    else:
        # saving the submission_ids_list to the text file
        for ID in submission_ids_list[:-1]:
            save_file.write("%s\n" % ID)
        save_file.write("%s" % submission_ids_list[-1])
        save_file.close()
        print('Saved succesfully to', save_name, '!')

In [6]:
def find_submissions(iterations, submissions_list=all_submissions, 
                     submission_ids_list=all_submission_ids, verbose=2):
    ''' Function for finding submissions within the CasualConversations subreddit using the 'random' method
    
    =========================== ===============================================
    Attribute                   Description
    =========================== ===============================================
    "iterations"                The amount of iterations this function has to
                                be run.
    "submissions_list"          To which list the submissions have to be
                                appended.
    "submission_ids_list"       To which list the submission IDs have to be
                                appended.
    "verbose"                   The level of verbose. 0 for no print 
                                statements; 1 for result and efficiency print 
                                statements; 2 also shows progress; 3 for most 
                                detail, but instead of progress bar, shows
                                iterations.
    =========================== ===============================================
    '''
    
    submissions_found = 0
    duplicates_found = 0
    progress_readout_threshold = 0
    
    if(verbose==2and iterations > 0): print('Progress : ')
    for iteration in range(iterations):
        if(verbose==2 and iterations > 0):
            if(iteration/float(iterations) * 100 >= progress_readout_threshold):
                print(str(round(iteration/float(iterations) * 100)) + '%')
                progress_readout_threshold += 4
        
        random_submission = r_CasualConversation.random()
        
        if(submission_ids_list.count(random_submission.id) == 0):
            submissions_list.append(random_submission)
            submission_ids_list.append(random_submission.id)
            if(verbose>=3):print('ID ' + str(random_submission.id) + ' was found at iteration ' + str(iteration))
            submissions_found += 1
        else:
            if(verbose>=3):print('Duplicate ID ' + str(random_submission.id) + ' was found.')
            duplicates_found += 1
    
    if(duplicates_found==0): submission_percentage = 100
    else: submission_percentage = round(submissions_found/float(iterations) * 100)

    if(verbose>=2):print('\n\n')
    if(verbose>=1):print('------------------------------------------------Result and Efficiency'+ 
                         '----------------------------------------------------------\n'
                         'Total amount of submissions found : ' + str(len(submission_ids_list)) + '\n\n' +
                         'Amount of submissions found over the ' + str(iterations) + ' iterations of this run : ' + 
                         str(submissions_found) + '\n' +
                         'Amount of duplicates found : ' + str(duplicates_found) + '\n' +
                         'Non-duplicate percentage : ' + str(submission_percentage) + '%'
                        )
    return submissions_list.copy(), submission_ids_list.copy()

In [7]:
r_CasualConversation = reddit.subreddit('CasualConversation')

In [10]:
# Check if there are certainly no duplicates
print('IDs in all_submission_ids : ' + str(len(all_submission_ids)))
print('IDs in all_submission_ids without duplicates : ' + str(len(set(all_submission_ids))))
print('No duplicates =', len(all_submission_ids) == len(set(all_submission_ids)))

IDs in all_submission_ids : 4149
IDs in all_submission_ids without duplicates : 4149
No duplicates = True


### Implementation of functions

In [9]:
all_submissions, all_submission_ids = load_submission_ids()

Loaded succesfully from submissions_CasualConversations.txt !


In [13]:
all_submissions[4000].selftext

'I guess I\'ve gained a little bit of a reputation around my town as being "the little gay boy from [insert large popular store]" and today I got recognized, but it definitely wasn\'t good. I was talking to someone, and they asked for a face pic so I obliged. Their response? "[Insert store] boy lol" I really tried to play it off with a "haha, now what about you?" And they kept pressing. "You work in the [insert one of the most busy departments in said store] don\'t you?" I again tried to play it off, but I was met with a large LOL and then promptly blocked. I understand if I\'m recognized, but this guy got to me. My self esteem was just obliterated and now I\'m afraid of getting recognized at work. '

In [ ]:
all_submissions, all_submission_ids = find_submissions(iterations=20,submission_ids_list=all_submission_ids,
                                                      submissions_list=all_submissions)

In [68]:
save_submission_ids(submission_ids_list=all_submission_ids)

Saved succesfully to submissions_CasualConversations.txt !


### Another approach that appends all new submissions while running

In [65]:
submissions_found = 0
duplicates_found = 0

for submission in r_CasualConversation.stream.submissions():
    if(all_submission_ids.count(submission.id) == 0):
        all_submissions.append(submission)
        all_submission_ids.append(submission.id)
        print('ID ' + str(submission.id) + ' was found.')
        submissions_found += 1
    else:
        print('Duplicate ID ' + str(submission.id) + ' was found.')
        duplicates_found += 1
    
    print('Submissions found : ', str(submissions_found))
    print('Duplicates found : ', str(duplicates_found))

Duplicate ID b053db was found.
Submissions found :  0
Duplicates found :  1
Duplicate ID b05dcj was found.
Submissions found :  0
Duplicates found :  2
Duplicate ID b052m0 was found.
Submissions found :  0
Duplicates found :  3
Duplicate ID b05tm5 was found.
Submissions found :  0
Duplicates found :  4
Duplicate ID b05uw8 was found.
Submissions found :  0
Duplicates found :  5
Duplicate ID b066f2 was found.
Submissions found :  0
Duplicates found :  6
Duplicate ID b06acq was found.
Submissions found :  0
Duplicates found :  7
Duplicate ID b06m7d was found.
Submissions found :  0
Duplicates found :  8
Duplicate ID b06u2l was found.
Submissions found :  0
Duplicates found :  9
Duplicate ID b06wll was found.
Submissions found :  0
Duplicates found :  10
Duplicate ID b06wt6 was found.
Submissions found :  0
Duplicates found :  11
Duplicate ID b07jqo was found.
Submissions found :  0
Duplicates found :  12
Duplicate ID b07nfp was found.
Submissions found :  0
Duplicates found :  13
Duplicat

ID b0gzdj was found.
Submissions found :  9
Duplicates found :  100
ID b0h61w was found.
Submissions found :  10
Duplicates found :  100
ID b0h6w5 was found.
Submissions found :  11
Duplicates found :  100
ID b0h77g was found.
Submissions found :  12
Duplicates found :  100
ID b0h9k7 was found.
Submissions found :  13
Duplicates found :  100
ID b0hd6e was found.
Submissions found :  14
Duplicates found :  100
ID b0hels was found.
Submissions found :  15
Duplicates found :  100
ID b0hh1x was found.
Submissions found :  16
Duplicates found :  100
ID b0hisp was found.
Submissions found :  17
Duplicates found :  100
ID b0ht7q was found.
Submissions found :  18
Duplicates found :  100
ID b0huzb was found.
Submissions found :  19
Duplicates found :  100
ID b0i229 was found.
Submissions found :  20
Duplicates found :  100
ID b0i8az was found.
Submissions found :  21
Duplicates found :  100
ID b0icix was found.
Submissions found :  22
Duplicates found :  100
ID b0iclf was found.
Submissions fo

KeyboardInterrupt: 